## Prepapre DB

In [20]:
import pandas as pd 
from tqdm import tqdm 
df = pd.read_csv('vbs23_meta.csv')
df.head()

,Unnamed: 0,id,location,camera_model,thumbnails,selected_frames,created_time,date,month,year,day_of_week,part_of_day,local_time,captions,width,height,ext,fps,index
0,Dapang_Jul2022_5_1,1,Dapang,DJI Osmo 3,information/thumbnails/Dapang_Jul2022/0005_000...,information/selected_frames/Dapang_Jul2022/000...,2022:08:27 07:48:37,2022-08-27,August,2022,Saturday,early morning,07:48,underwater view of a coral reef.,1920,1080,mp4,30.0,Dapang_Jul2022_5_1
1,Dapang_Jul2022_5_2,2,Dapang,DJI Osmo 3,information/thumbnails/Dapang_Jul2022/0005_000...,information/selected_frames/Dapang_Jul2022/000...,2022:08:27 07:48:37,2022-08-27,August,2022,Saturday,early morning,07:48,the reef is a bit smaller than the ones we saw.,1920,1080,mp4,30.0,Dapang_Jul2022_5_2
2,Dapang_Jul2022_5_3,3,Dapang,DJI Osmo 3,information/thumbnails/Dapang_Jul2022/0005_000...,information/selected_frames/Dapang_Jul2022/000...,2022:08:27 07:48:37,2022-08-27,August,2022,Saturday,early morning,07:48,the reef is a bit more shallow than the beach.,1920,1080,mp4,30.0,Dapang_Jul2022_5_3
3,Dapang_Jul2022_5_4,4,Dapang,DJI Osmo 3,information/thumbnails/Dapang_Jul2022/0005_000...,information/selected_frames/Dapang_Jul2022/000...,2022:08:27 07:48:37,2022-08-27,August,2022,Saturday,early morning,07:48,a small group of fish swim.,1920,1080,mp4,30.0,Dapang_Jul2022_5_4
4,Dapang_Jul2022_5_5,5,Dapang,DJI Osmo 3,information/thumbnails/Dapang_Jul2022/0005_000...,information/selected_frames/Dapang_Jul2022/000...,2022:08:27 07:48:37,2022-08-27,August,2022,Saturday,early morning,07:48,a small group of fish swim around a coral reef.,1920,1080,mp4,30.0,Dapang_Jul2022_5_5


In [21]:
# pd.Timestamp.max

In [22]:
# df['created_time'] = pd.to_datetime(df['created_time'])

## Create index in Elasticsearch

In [23]:
from elasticsearch.helpers import bulk
from elasticsearch import Elasticsearch, RequestsHttpConnection

es = Elasticsearch(['http://0.0.0.0:9200'], timeout=100, connection_class=RequestsHttpConnection, http_auth=('elastic', '123456'), use_ssl=False, verify_certs=False)

In [24]:
es.indices.create(index='vbs23_db', ignore=400)
es.indices.get_alias("*")

/tmp/ipykernel_3967146/1932975029.py:2: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.get_alias("*")


{'vbs23_db': {'aliases': {}}}

In [25]:

for i, row in df.iterrows():
    print(row['index'])
    data = row.to_dict()
    data.pop('index', None)
    print(data)
    break

Dapang_Jul2022_5_1
{'Unnamed: 0': 'Dapang_Jul2022_5_1', 'id': 1, 'location': 'Dapang', 'camera_model': 'DJI Osmo 3', 'thumbnails': 'information/thumbnails/Dapang_Jul2022/0005_00001.jpg', 'selected_frames': 'information/selected_frames/Dapang_Jul2022/0005_00001.jpg', 'created_time': '2022:08:27 07:48:37', 'date': '2022-08-27', 'month': 'August', 'year': 2022, 'day_of_week': 'Saturday', 'part_of_day': 'early morning', 'local_time': '07:48', 'captions': 'underwater view of a coral reef.', 'width': 1920, 'height': 1080, 'ext': 'mp4', 'fps': 30.0}


In [26]:
def gen_data():
    for i, row in tqdm(df.iterrows(), total=len(df)):
        data = row.to_dict()
        data.pop('index', None)
        yield {
            "_index": 'vbs23_db',
            "_id": row['index'],
            "_source": data
        }
bulk(es, gen_data())

100%|██████████| 43797/43797 [00:05<00:00, 8322.73it/s]


(43797, [])

In [27]:
# Delete index
# es.indices.delete(index='vbs23_db')